In [31]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('ha_data.csv', sep=';')
df = df.drop('order_id', axis=1).reset_index(names='order_id')

In [4]:
df

,order_id,page_id,product,site_version,time,title,user
0,0,3.0,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,1,3699687.0,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,2,14.0,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,3,10362176.0,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,4,14.0,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1
...,...,...,...,...,...,...,...
8471221,8471221,24897184.0,accessories,desktop,2017-05-23 14:07:00,banner_show,user_4254616
8471222,8471222,24897190.0,clothes,mobile,2017-05-28 08:10:20,banner_show,user_4254617
8471223,8471223,24897203.0,sports_nutrition,mobile,2017-05-20 09:20:50,banner_show,user_4254618
8471224,8471224,24897205.0,sneakers,mobile,2017-05-28 19:25:42,banner_show,user_4254619


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8471226 entries, 0 to 8471225
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   order_id      int64  
 1   page_id       float64
 2   product       object 
 3   site_version  object 
 4   time          object 
 5   title         object 
 6   user          object 
dtypes: float64(1), int64(1), object(5)
memory usage: 452.4+ MB


Поменяем тип данных в колонке page_id на int, если там нет дробных значений, для удобства будущих подсчетов.
Значения покупок установим 0

In [6]:
(df['page_id'] % 1).value_counts()

page_id
0.0    8222504
Name: count, dtype: int64

In [62]:
tt = df
tt['page_id'].fillna(0, inplace=True)
tt['page_id'] = tt['page_id'].astype(int)
tt

C:\Users\Qwert\AppData\Local\Temp\ipykernel_28784\1764642494.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tt['page_id'].fillna(0, inplace=True)


,order_id,page_id,product,site_version,time,title,user
0,1,3,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,1,3699687,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,1,14,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,1,10362176,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,0,14,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1
...,...,...,...,...,...,...,...
8471221,1,24897184,accessories,desktop,2017-05-23 14:07:00,banner_show,user_4254616
8471222,1,24897190,clothes,mobile,2017-05-28 08:10:20,banner_show,user_4254617
8471223,1,24897203,sports_nutrition,mobile,2017-05-20 09:20:50,banner_show,user_4254618
8471224,1,24897205,sneakers,mobile,2017-05-28 19:25:42,banner_show,user_4254619


Теперь поменяем значения в колонке order_id. 0 - Клик, 1 - показ.

In [8]:
tt.loc[tt['title'] == 'banner_click', 'order_id'] = 0
tt.loc[tt['title'] == 'banner_show', 'order_id'] = 1
tt['order_id'].value_counts()

order_id
1          7393319
0           829185
6223273          1
5508724          1
5507694          1
            ...   
2802342          1
2802378          1
2802385          1
2802414          1
8471159          1
Name: count, Length: 248724, dtype: int64

Узнаем за какой промежуток времени у нас есть информация

In [15]:
tt['time'] = pd.to_datetime(tt['time'])
tt['time'].max() - tt['time'].min()

Timedelta('150 days 23:59:55')

Посмотрим сколько и каких событий произошло за этот промежуток времени. Заметим, что значения не отличаются от колонки order_id.

In [9]:
tt['title'].value_counts()

title
banner_show     7393319
banner_click     829185
order            248722
Name: count, dtype: int64

Теперь узнаем, сколько событий для каждого продукта произошло всего

In [17]:
tt.groupby('title')['product'].value_counts().reset_index()

,title,product,count
0,banner_click,clothes,220595
1,banner_click,sneakers,182597
2,banner_click,sports_nutrition,144918
3,banner_click,company,144821
4,banner_click,accessories,136254
5,banner_show,company,1580238
6,banner_show,sports_nutrition,1465970
7,banner_show,clothes,1453128
8,banner_show,sneakers,1453029
9,banner_show,accessories,1440954


Предположим, что в случае продажи баннера конверсия будет такой же, как и сейчас при рекламе всей компании.
Найдем её

In [45]:
round(tt[(tt['title'] == 'banner_click') & (tt['product'] == 'company')].shape[0] / tt[(tt['title'] == 'banner_show') & (tt['product'] == 'company')].shape[0], 3) * 100

9.2

In [46]:
round(144821 / 1580238, 3) * 100
#Сверим со значением в таблице выше

9.2

Теперь найдем общее количество кликов на баннер при такой конверсии

In [48]:
tmp = (tt[tt['title'] == 'banner_show'].shape[0] * 0.092)
np.floor(tmp)

680185.0

Теперь посмотрим конверсию в продажи товаров. Так как у нас нет сведений о цене конкретного товара - предположим, что цена у них одинаковая.  

In [56]:
round(tt[(tt['title'] == 'order')].shape[0] / tt[(tt['title'] == 'banner_show')].shape[0] * 100, 1)

3.4

In [58]:
round((112715 + 67719 + 44551 + 23737) / (1580238 + 1465970 + 1453128 + 1453029 + 1440954) * 100, 1)
#Сверим со значением в таблице выше

3.4

In [60]:
tmp2 = (1580238 + 1465970 + 1453128 + 1453029 + 1440954) * 0.034
np.floor(tmp2)

251372.0

отношение кликов на компанию к количеству заказов

In [61]:
tmp / tmp2

2.705882352941176

Получается, для того, чтобы получать столько же прибыли, что и с продажи товаров и рекламы компании необходимо, чтобы цена клика была примерно в 2.7 раз меньше, чем прибыль с продажи товара.
 

Зная эту информацию - я бы посчитал конверсию из показа в покупку для каждого типа товара(на примере ячейки ниже). Далее посчитал бы суммарную прибыль с продажи всех товаров и смог бы точнее дать ответ на поставленный вопрос

In [65]:
#конверсия из просмотра в заказы на примере кроссовок
round(tt[(tt['title'] == 'order') & (tt['product'] == 'sneakers')].shape[0] / tt[(tt['title'] == 'banner_click') & (tt['product'] == 'sneakers')].shape[0] * 100, 1)

37.1